# Using Hugging Face with Sentimental Analysis

In [1]:
from transformers import pipeline
import requests
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import re
import datetime 
from konlpy.tag import Hannanum
from config import * # config안의 정보 가져오기

In [2]:
def get_today_stock_news_data_list (ticker_name):

    #start_time = datetime.datetime.today().strftime("%Y-%m-%dT00:00:00")
    #end_time = datetime.datetime.today().strftime("%Y-%m-%dT23:59:59") 이건 오늘 날짜로 따질때 이야기

    start_time = "2022-04-05T00:00:00"
    end_time = "2022-04-05T23:59:59"


    headers = {
    'Content-Type': 'application/json'
    }

    url = f"{ELASTIC_SEARCH_URL}/news/_count"

    query=f'''{{
    "query": {{
      "bool": {{
        "must": [
          {{
            "match": {{
              "body": "{ticker_name}"
            }}
          }},
          {{
            "range": {{
              "created_at": {{
               "gte": "{start_time}",
               "lte": "{end_time}"
              }}
            }}
          }}
        ]
      }}
    }}
    }}'''

    count = int(requests.get(url, data=query.encode('utf-8'), headers=headers, auth = ELASTIC_SEARCH_AUTH).json()['count'])

    if count != 0:

      url = f"{ELASTIC_SEARCH_URL}/news/_search"

      query=f'''{{
      "size": {count},
      "query": {{
        "bool": {{
          "must": [
            {{
              "match": {{
                "body": "{ticker_name}"
              }}
            }},
            {{
              "range": {{
                "created_at": {{
                "gte": "{start_time}",
                "lte": "{end_time}"
                }}
              }}
            }}
          ]
        }}
      }}
      }}'''  

      resp = requests.get(url, data=query.encode('utf-8'), headers=headers, auth = ELASTIC_SEARCH_AUTH).json()

      resp = resp['hits']['hits']
    
    else:

      resp = []


    return resp

def clean_doc (doc):
    
    doc = re.sub(r'[^\wㄱ-ㅎ가-힣]', ' ', doc)
    doc = re.sub('  ', ' ', doc)
    doc = re.sub('  ', ' ', doc)

    return doc

def title_extract(doc):
    
    return doc['_source']['title']
    
def body_extract (doc):

    return doc['_source']['body']

In [3]:
classifier = pipeline('sentiment-analysis', model='snunlp/KR-FinBert-SC')

In [11]:
stock1_doc_list = get_today_stock_news_data_list("삼성전자")
title_list = [title_extract(x) for x in stock1_doc_list]

title_list[0:5]
body_list = []

for i in stock1_doc_list[0:10]:
    a = body_extract(i)
    a = clean_doc(a)
    body_list.append(a)

In [13]:
body_list[0]

' 반도체 수퍼사이클 장기 상승장 이라고 해서 샀는데 계속 물타다가 대주주 되겠어요 이번에 실적 발표를 보고 추매와 손절 중에 선택하려고요 50대 투자자 이모씨 오는 7일 삼성전자의 1분기 1 3월 실적 발표일을 앞두고 500만 삼성전자 소액 주주들이 초긴장 모드다 삼성전자 주가가 5일 6만9200원까지 주저앉은 가운데 1분기 실적 발표를 보면서 향후 투자의 방향을 결정하려는 것이다 이날 한국거래소에 따르면 올해 개인 투자자들은 삼성전자 주식을 6조5000억원 어치 사모았다 삼성전자 우선주까지 합하면 7조원이 넘는다 같은 기간 연기금 등 기관은 삼성전자 주식을 5조원 넘게 팔아 치웠고 외국인도 1조원 넘게 순매도했다 개인들은 삼성전자 주가가 6만원대로 떨어진 3월에만 3조8000억원 어치 순매수했다 나머지 2 10위 종목들의 순매수 금액을 다 합해도 삼성전자 한 종목에 미치지 못한다 그래픽 이연주 조선디자인랩 기자 개인 투자자들이 주목하고 있는 삼성전자의 1분기 실적 전망은 어떨까 이달 초 금융정보업체 에프앤가이드가 집계한 자료에 따르면 삼성전자 1분기 실적에 대한 증권사 전망치 컨센서스 는 매출액 75조823억원 영업이익은 13조283억원이다 증권가 예상이 맞다면 삼성전자는 1분기 기준으로 매출 70조원 시대를 처음 열게 된다 1분기 호실적 발표가 예상되고 있지만 주가 흐름은 영 신통치 않다 지난 4일 삼성전자 종가는 6만9300원 올해 들어서만 12 하락했다 작년 초만 해도 반도체 수퍼사이클 전망이 나오면서 주가는 9만원을 돌파해 10만원 턱밑까지 올랐지만 1년여 만에 6만원대로 밀려 버렸다 편득현 NH투자증권 WM마스터즈 전문위원은 삼성전자는 한국 증시에 대한 비중을 낮추는 연기금과 외국인들의 집중 타깃이 되고 있다 면서 올해 실적 개선폭이 전년 대비 크지 않다는 이유로 매도세가 이어지는 모습 이라고 말했다 편 위원은 이어 하지만 환율 효과로 실적이 예상보다 더 좋게 나올 수 있고 두 매매 주체가 연일 큰 규모로 매도해도 주가가 크게 빠지지 않는다는 점에

In [18]:
result = classifier(title_list)

In [4]:
classifier(" 이제는 폴더블폰 2Q 및 하반기에 고성장 이제는 폴더블폰 2Q 및 하반기에 고성장 이제는 폴더블폰 2Q 및 하반기에 고성장 ")

[{'label': 'positive', 'score': 0.999484658241272}]

In [5]:
classifier(" 신규 수출 계약 기대감 유효 신규 수출 계약 기대감 유효 떨어져가는 원전 일감 신사업 진출로 인한 과도기 연내 원전 이벤트 수출 에기본과 전기본 밸류에이션 부담은 여전 원전 관련 이벤트 전후 적극적인 Trading 필요 ")

[{'label': 'positive', 'score': 0.9998816251754761}]

In [19]:
result[0:10]

[{'label': 'neutral', 'score': 0.9998371601104736},
 {'label': 'neutral', 'score': 0.9985515475273132},
 {'label': 'neutral', 'score': 0.9998544454574585},
 {'label': 'neutral', 'score': 0.9998384714126587},
 {'label': 'neutral', 'score': 0.9996901750564575},
 {'label': 'neutral', 'score': 0.9998394250869751},
 {'label': 'negative', 'score': 0.6307186484336853},
 {'label': 'neutral', 'score': 0.9998999834060669},
 {'label': 'neutral', 'score': 0.9932652711868286},
 {'label': 'neutral', 'score': 0.9998489618301392}]

In [24]:
result[136]

{'label': 'neutral', 'score': 0.5417889952659607}

In [27]:
result[0]['score']

0.9998371601104736

In [34]:
title_list = title_list[0:20]

for i in range(0, len(title_list)-1):
    print(f"이 뉴스는 {round(result[i]['score'] * 100, 2)}% 확률로 {result[i]['label']} 입니다.")
    print(f"제목 : {title_list[i]}\n")

이 뉴스는 99.98% 확률로 neutral 입니다.
제목 : 3년 전 산 삼성전자 지분이 지금은… 블랙록의 돌부처 투자법 [왕개미연구소]

이 뉴스는 99.86% 확률로 neutral 입니다.
제목 : 상속세 때문에 홍라희가 판 삼성전자 지분, '이곳'이 가져갔다

이 뉴스는 99.99% 확률로 neutral 입니다.
제목 : 임직원과 소통 나선 한종희, 부회장 된 후 첫 이메일 보낸 사연

이 뉴스는 99.98% 확률로 neutral 입니다.
제목 : 'JH 입니다' 한종희 삼성 부회장, 임직원과 이메일 소통 나섰다

이 뉴스는 99.97% 확률로 neutral 입니다.
제목 : 삼성전자 부회장 'JH'의 소통…임직원 건의 직접 듣고 바꿨다

이 뉴스는 99.98% 확률로 neutral 입니다.
제목 : 삼성 'JH' 한종희 부회장, 임직원에 이메일 보내 '소통 행보'

이 뉴스는 63.07% 확률로 negative 입니다.
제목 : JP모간, 홍라희 삼전 지분 매입뒤 1900건 넘게 거래…또 '단타'?

이 뉴스는 99.99% 확률로 neutral 입니다.
제목 : 코스피, 강보합 마감…2760선 턱밑

이 뉴스는 99.33% 확률로 neutral 입니다.
제목 : 반도체 업계 또 놀라겠네…DB하이텍 파격 보상 "신입 연봉 7200만원"

이 뉴스는 99.98% 확률로 neutral 입니다.
제목 : 삼성 한종희 부회장 '소통행보'…건의사항에 이메일로 화답

이 뉴스는 99.98% 확률로 neutral 입니다.
제목 : `JH입니다` 한종희 삼성 부회장, 임직원 소통 강화 행보

이 뉴스는 99.09% 확률로 neutral 입니다.
제목 : 삼성전자, 젊은 석학들에 미래기술 맡겼다

이 뉴스는 98.71% 확률로 neutral 입니다.
제목 : "JH입니다"…한종희, 삼성 직원들 건의사항에 '이메일 화답'

이 뉴스는 92.17% 확률로 negative 입니다.
제목 : 삼전·SK하이닉스, '큰손' 탈출 러시…'반도체의 봄' 힘들 듯

이 뉴스는 99.9

In [53]:
body_result = []

for i in body_list:
    tmp = classifier(i[0:1138])[0]
    body_result.append(tmp)

body_result[0]

{'label': 'positive', 'score': 0.5799351930618286}

In [54]:
body_result

[{'label': 'positive', 'score': 0.5799351930618286},
 {'label': 'neutral', 'score': 0.9999164342880249},
 {'label': 'neutral', 'score': 0.9999048709869385},
 {'label': 'neutral', 'score': 0.9998095631599426},
 {'label': 'neutral', 'score': 0.9998488426208496},
 {'label': 'neutral', 'score': 0.9998942613601685},
 {'label': 'positive', 'score': 0.8094359636306763},
 {'label': 'neutral', 'score': 0.8014070987701416},
 {'label': 'neutral', 'score': 0.9993265867233276},
 {'label': 'neutral', 'score': 0.9998965263366699}]

In [55]:
for i in range(0, len(body_list)-1):
    print(f"이 뉴스는 {round(body_result[i]['score'] * 100, 2)}% 확률로 {body_result[i]['label']} 입니다.")
    print(f"본문 : {body_list[i]}\n")

이 뉴스는 57.99% 확률로 positive 입니다.
본문 :  반도체 수퍼사이클 장기 상승장 이라고 해서 샀는데 계속 물타다가 대주주 되겠어요 이번에 실적 발표를 보고 추매와 손절 중에 선택하려고요 50대 투자자 이모씨 오는 7일 삼성전자의 1분기 1 3월 실적 발표일을 앞두고 500만 삼성전자 소액 주주들이 초긴장 모드다 삼성전자 주가가 5일 6만9200원까지 주저앉은 가운데 1분기 실적 발표를 보면서 향후 투자의 방향을 결정하려는 것이다 이날 한국거래소에 따르면 올해 개인 투자자들은 삼성전자 주식을 6조5000억원 어치 사모았다 삼성전자 우선주까지 합하면 7조원이 넘는다 같은 기간 연기금 등 기관은 삼성전자 주식을 5조원 넘게 팔아 치웠고 외국인도 1조원 넘게 순매도했다 개인들은 삼성전자 주가가 6만원대로 떨어진 3월에만 3조8000억원 어치 순매수했다 나머지 2 10위 종목들의 순매수 금액을 다 합해도 삼성전자 한 종목에 미치지 못한다 그래픽 이연주 조선디자인랩 기자 개인 투자자들이 주목하고 있는 삼성전자의 1분기 실적 전망은 어떨까 이달 초 금융정보업체 에프앤가이드가 집계한 자료에 따르면 삼성전자 1분기 실적에 대한 증권사 전망치 컨센서스 는 매출액 75조823억원 영업이익은 13조283억원이다 증권가 예상이 맞다면 삼성전자는 1분기 기준으로 매출 70조원 시대를 처음 열게 된다 1분기 호실적 발표가 예상되고 있지만 주가 흐름은 영 신통치 않다 지난 4일 삼성전자 종가는 6만9300원 올해 들어서만 12 하락했다 작년 초만 해도 반도체 수퍼사이클 전망이 나오면서 주가는 9만원을 돌파해 10만원 턱밑까지 올랐지만 1년여 만에 6만원대로 밀려 버렸다 편득현 NH투자증권 WM마스터즈 전문위원은 삼성전자는 한국 증시에 대한 비중을 낮추는 연기금과 외국인들의 집중 타깃이 되고 있다 면서 올해 실적 개선폭이 전년 대비 크지 않다는 이유로 매도세가 이어지는 모습 이라고 말했다 편 위원은 이어 하지만 환율 효과로 실적이 예상보다 더 좋게 나올 수 있고 두 매매 주